In [1]:
# https://www.tensortrade.org/en/latest/examples/setup_environment_tutorial.html
import ta

import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.instruments import USD, BTC, ETH, LTC
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order


In [2]:
cdd = CryptoDataDownload()

bitfinex_data = pd.concat([
    cdd.fetch("Bitfinex", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Bitfinex", "USD", "ETH", "1h").add_prefix("ETH:")
], axis=1)

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Bitstamp", "USD", "LTC", "1h").add_prefix("LTC:")
], axis=1)

In [3]:
bitfinex = Exchange("bitfinex", service=execute_order)(
    Stream.source(list(bitfinex_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitfinex_data['ETH:close']), dtype="float").rename("USD-ETH")
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitstamp_data['LTC:close']), dtype="float").rename("USD-LTC")
)

In [4]:
# Add all features for bitstamp BTC & ETH
bitfinex_btc = bitfinex_data.loc[:, [name.startswith("BTC") for name in bitfinex_data.columns]]
bitfinex_eth = bitfinex_data.loc[:, [name.startswith("ETH") for name in bitfinex_data.columns]]

ta.add_all_ta_features(
    bitfinex_btc,
    colprefix="BTC:",
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)


with NameSpace("bitfinex"):
    bitfinex_streams = [
        Stream.source(list(bitfinex_btc[c]), dtype="float").rename(c) for c in bitfinex_btc.columns
    ]
    bitfinex_streams += [
        Stream.source(list(bitfinex_eth[c]), dtype="float").rename(c) for c in bitfinex_eth.columns
    ]


# Add all features for bitstamp BTC & LTC
bitstamp_btc = bitstamp_data.loc[:, [name.startswith("BTC") for name in bitstamp_data.columns]]  
bitstamp_ltc = bitstamp_data.loc[:, [name.startswith("LTC") for name in bitstamp_data.columns]]

ta.add_all_ta_features(
    bitstamp_ltc,
    colprefix="LTC:",
    **{k: "LTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

with NameSpace("bitstamp"):
    bitstamp_streams = [
        Stream.source(list(bitstamp_btc[c]), dtype="float").rename(c) for c in bitstamp_btc.columns
    ]
    bitstamp_streams += [
        Stream.source(list(bitstamp_ltc[c]), dtype="float").rename(c) for c in bitstamp_ltc.columns
    ]


feed = DataFeed(bitfinex_streams + bitstamp_streams)


/home/klemen/.local/lib/python3.8/site-packages/ta/wrapper.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_adi"] = AccDistIndexIndicator(
/home/klemen/.local/lib/python3.8/site-packages/ta/wrapper.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_obv"] = OnBalanceVolumeIndicator(
/home/klemen/.local/lib/python3.8/site-packages/ta/wrapper.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [5]:
feed.next()

{'bitfinex:/BTC:date': Timestamp('2018-05-15 06:00:00'),
 'bitfinex:/BTC:unix': 1526364000,
 'bitfinex:/BTC:open': 8723.8,
 'bitfinex:/BTC:high': 8793.0,
 'bitfinex:/BTC:low': 8714.9,
 'bitfinex:/BTC:close': 8739.0,
 'bitfinex:/BTC:volume': 8988053.53,
 'bitfinex:/BTC:volume_adi': -3441008.969871901,
 'bitfinex:/BTC:volume_obv': 8988053.53,
 'bitfinex:/BTC:volume_cmf': nan,
 'bitfinex:/BTC:volume_fi': nan,
 'bitfinex:/BTC:volume_mfi': nan,
 'bitfinex:/BTC:volume_em': nan,
 'bitfinex:/BTC:volume_sma_em': nan,
 'bitfinex:/BTC:volume_vpt': -813501.9453160722,
 'bitfinex:/BTC:volume_nvi': 1000.0,
 'bitfinex:/BTC:volume_vwap': nan,
 'bitfinex:/BTC:volatility_atr': 0.0,
 'bitfinex:/BTC:volatility_bbm': nan,
 'bitfinex:/BTC:volatility_bbh': nan,
 'bitfinex:/BTC:volatility_bbl': nan,
 'bitfinex:/BTC:volatility_bbw': nan,
 'bitfinex:/BTC:volatility_bbp': nan,
 'bitfinex:/BTC:volatility_bbhi': 0.0,
 'bitfinex:/BTC:volatility_bbli': 0.0,
 'bitfinex:/BTC:volatility_kcc': nan,
 'bitfinex:/BTC:volat

In [6]:
portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 10 * BTC),
    Wallet(bitfinex, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 3 * LTC),
])


In [7]:
env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="simple",
    feed=feed,
    window_size=15,
    enable_logger=False
)

In [8]:
env.observer.feed.next()



{'internal': {'bitfinex:/USD-BTC': 8739.0,
  'bitfinex:/USD-ETH': 733.04,
  'bitfinex:/USD:/free': 10000.0,
  'bitfinex:/USD:/locked': 0.0,
  'bitfinex:/USD:/total': 10000.0,
  'bitfinex:/BTC:/free': 10.0,
  'bitfinex:/BTC:/locked': 0.0,
  'bitfinex:/BTC:/total': 10.0,
  'bitfinex:/BTC:/worth': 87390.0,
  'bitfinex:/ETH:/free': 5.0,
  'bitfinex:/ETH:/locked': 0.0,
  'bitfinex:/ETH:/total': 5.0,
  'bitfinex:/ETH:/worth': 3665.2,
  'bitstamp:/USD-BTC': 8740.99,
  'bitstamp:/USD-LTC': 147.2,
  'bitstamp:/USD:/free': 1000.0,
  'bitstamp:/USD:/locked': 0.0,
  'bitstamp:/USD:/total': 1000.0,
  'bitstamp:/BTC:/free': 5.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 5.0,
  'bitstamp:/BTC:/worth': 43704.95,
  'bitstamp:/LTC:/free': 3.0,
  'bitstamp:/LTC:/locked': 0.0,
  'bitstamp:/LTC:/total': 3.0,
  'bitstamp:/LTC:/worth': 441.59999999999997,
  'net_worth': 146201.75},
 'external': {'bitfinex:/BTC:date': Timestamp('2018-05-15 06:00:00'),
  'bitfinex:/BTC:unix': 1526364000,
  'bit

In [9]:
!python3 -m pip install git+https://github.com/tensortrade-org/tensortrade.git

  Cloning https://github.com/tensortrade-org/tensortrade.git to /tmp/pip-req-build-rti5vdv8
  Running command git clone -q https://github.com/tensortrade-org/tensortrade.git /tmp/pip-req-build-rti5vdv8


  Created wheel for tensortrade: filename=tensortrade-1.0.2.dev0-py3-none-any.whl size=133825 sha256=3169d53d40b31ccc804df18235dab4f3240b9b48b9355e7960236d80ff12bb18
  Stored in directory: /tmp/pip-ephem-wheel-cache-80mo5z08/wheels/56/40/cd/062610cd30890c4d916f4d2141799782cfe27a6357233016f4
Successfully built tensortrade


In [10]:
import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import DQNAgent

In [11]:
%matplotlib inline

In [12]:
cdd = CryptoDataDownload()

data = cdd.fetch("Bitfinex", "USD", "BTC", "1h")

In [13]:
def rsi(price: Stream[float], period: float) -> Stream[float]:
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Stream[float], fast: float, slow: float, signal: float) -> Stream[float]:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream.source(list(data[c]), dtype="float").rename(data[c].name)
    features += [s]

cp = Stream.select(features, lambda s: s.name == "close")

features = [
    cp.log().diff().rename("lr"),
    rsi(cp, period=20).rename("rsi"),
    macd(cp, fast=10, slow=50, signal=5).rename("macd")
]

feed = DataFeed(features)
feed.compile()

In [14]:
for i in range(5):
    print(feed.next())

{'lr': nan, 'rsi': nan, 'macd': 0.0}
{'lr': 0.00045761355334761333, 'rsi': 100.0, 'macd': 0.3802733214494462}
{'lr': -0.0022099203461092287, 'rsi': 16.45021645021696, 'macd': -1.2850831900864756}
{'lr': -0.001789832853125617, 'rsi': 9.61512851245181, 'macd': -3.5410157838872425}
{'lr': 0.00872379407148216, 'rsi': 71.2075575333076, 'macd': 2.810321575123835}


/usr/local/lib/python3.8/dist-packages/tensortrade-1.0.2.dev0-py3.8.egg/tensortrade/feed/core/operators.py:171: RuntimeWarning: divide by zero encountered in true_divide
  return self.op(self.inputs[0].value, self.inputs[1].value)


In [15]:
bitfinex = Exchange("bitfinex", service=execute_order)(
    Stream.source(list(data["close"]), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 10 * BTC)
])


renderer_feed = DataFeed([
    Stream.source(list(data["date"])).rename("date"),
    Stream.source(list(data["open"]), dtype="float").rename("open"),
    Stream.source(list(data["high"]), dtype="float").rename("high"),
    Stream.source(list(data["low"]), dtype="float").rename("low"),
    Stream.source(list(data["close"]), dtype="float").rename("close"),
    Stream.source(list(data["volume"]), dtype="float").rename("volume")
])


env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer_feed=renderer_feed,
    renderer=default.renderers.PlotlyTradingChart(),
    window_size=20
)

In [16]:
env.observer.feed.next()


{'internal': {'bitfinex:/USD-BTC': 8739.0,
  'bitfinex:/USD:/free': 10000.0,
  'bitfinex:/USD:/locked': 0.0,
  'bitfinex:/USD:/total': 10000.0,
  'bitfinex:/BTC:/free': 10.0,
  'bitfinex:/BTC:/locked': 0.0,
  'bitfinex:/BTC:/total': 10.0,
  'bitfinex:/BTC:/worth': 87390.0,
  'net_worth': 97390.0},
 'external': {'lr': nan, 'rsi': nan, 'macd': 0.0},
 'renderer': {'date': Timestamp('2018-05-15 06:00:00'),
  'open': 8723.8,
  'high': 8793.0,
  'low': 8714.9,
  'close': 8739.0,
  'volume': 8988053.53}}

In [19]:
agent = DQNAgent(env)

agent.train(n_steps=200, n_episodes=200, save_path="agents/")

FigureWidget({
    'data': [{'close': array([8739. , 8743. , 8723.7, ..., 7905.9, 7897.3, 7877.4]),
          …

FigureWidget({
    'data': [{'close': array([8739.  , 8743.  , 8723.7 , 8708.1 , 8784.4 , 8755.2 , 8762.  , 87…

FigureWidget({
    'data': [{'close': array([8739.  , 8743.  , 8723.7 , 8708.1 , 8784.4 , 8755.2 , 8762.  , 87…

FigureWidget({
    'data': [{'close': array([8739.  , 8743.  , 8723.7 , 8708.1 , 8784.4 , 8755.2 , 8762.  , 87…

-85585895.40555248